In [14]:
using SCS
using DiffOpt
using Dualization
using MathOptInterface
using MatrixOptInterface
# using MathOptSetDistances
# using LinearAlgebra
using Test
# using SparseArrays

const MOI = MathOptInterface
const MOIT = MathOptInterface.Test
const MOIU = MathOptInterface.Utilities
# const MOSD = MathOptSetDistances
const MatOI = MatrixOptInterface;

┌ Info: Precompiling Dualization [191a621a-6537-11e9-281d-650236a99e60]
└ @ Base loading.jl:1260


In [15]:
model = diff_optimizer(SCS.Optimizer)

δ = √(1 + (3*√2+2)*√(-116*√2+166) / 14) / 2
ε = √((1 - 2*(√2-1)*δ^2) / (2-√2))
y2 = 1 - ε*δ
y1 = 1 - √2*y2
obj = y1 + y2/2
k = -2*δ/ε
x2 = ((3-2obj)*(2+k^2)-4) / (4*(2+k^2)-4*√2)
α = √(3-2obj-4x2)/2
β = k*α

X = MOI.add_variables(model, 6)
x = MOI.add_variables(model, 3)

vov = MOI.VectorOfVariables(X)
cX = MOI.add_constraint(model, MOI.VectorAffineFunction{Float64}(vov), MOI.PositiveSemidefiniteConeTriangle(3))
cx = MOI.add_constraint(model, MOI.VectorAffineFunction{Float64}(MOI.VectorOfVariables(x)), MOI.SecondOrderCone(3))

c1 = MOI.add_constraint(
    model, 
    MOI.VectorAffineFunction(
        MOI.VectorAffineTerm.(1:1, MOI.ScalarAffineTerm.([1., 1., 1., 1.], [X[1], X[3], X[end], x[1]])), 
        [-1.0]
    ), 
    MOI.Zeros(1)
)
c2 = MOI.add_constraint(
    model, 
    MOI.VectorAffineFunction(
        MOI.VectorAffineTerm.(1:1, MOI.ScalarAffineTerm.([1., 2, 1, 2, 2, 1, 1, 1], [X; x[2]; x[3]])), 
        [-0.5]
    ), 
    MOI.Zeros(1)
)

objXidx = [1:3; 5:6]
objXcoefs = 2*ones(5)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([objXcoefs; 1.0], [X[objXidx]; x[1]]), 0.0))
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

sol = MOI.optimize!(model)

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 21, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 9, constraints m = 11
Cones:	primal zero / dual free vars: 2
	soc vars: 3, soc blks: 1
	sd vars: 6, sd blks: 1
Setup time: 1.57e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.04e+00  6.01e+00  7.35e-01  2.11e+00  1.65e+01  0.00e+00  1.37e-04 
    60| 6.47e-10  7.11e-10  1.70e-09  7.06e-01  7.06e-01  3.93e-16  1.07e-03 
---------------------------------------

SCS.MOISolution(1, "Solved", [0.217251238770051, -0.2599704147186366, 0.31108967252616243, 0.2172512383948794, -0.25997041471863536, 0.21725123877005162, 0.25440785083602735, 0.17989351643122645, 0.17989351643122645], [0.5447582105062065, 0.3219045549528856, 0.45524178943652027, -0.3219045563900619, -0.3219045563900619, 1.133337233849821, 0.9589717750123417, -0.4552417892860504, 1.1333372326447106, 0.958971775012335, 1.1333372338498173], [-5.8304524274385224e-18, 1.672022895212547e-17, 0.25440785071939953, 0.17989351643078233, 0.17989351643078233, 0.21725123843399946, -0.36765368641915924, 0.30723964791089897, 0.31108967225522005, -0.3676536864191592, 0.2172512384339987], 0.7057104920940137, 0.7057104879826493, 0.0, 0.001225041, 60)

In [9]:
MOI.get(model, MOI.VariablePrimal(), model.var_idx)

9-element Array{Float64,1}:
  0.217251238770051
 -0.2599704147186366
  0.31108967252616243
  0.2172512383948794
 -0.25997041471863536
  0.21725123877005162
  0.25440785083602735
  0.17989351643122645
  0.17989351643122645

In [11]:
model.dual_optimal

4-element Array{Union{Float64, Array{Float64,1}},1}:
 [1.133337233849821, 0.6780954450777269, 1.1333372326447106, -0.3219045562836636, 0.6780954450777222, 1.1333372338498173]
 [0.45524178943652027, -0.3219045563900619, -0.3219045563900619]
 [0.5447582105062065]
 [0.3219045549528856]

In [18]:
joint_object    = dualize(model)
dual_model_like = joint_object.dual_model # this is MOI.ModelLike, not an MOI.AbstractOptimizer; can't call optimizer on it
primal_dual_map = joint_object.primal_dual_map;

# copy the dual model objective, constraints, and variables to an optimizer
dual_model = MOI.instantiate(SCS.Optimizer, with_bridge_type=Float64)
MOI.copy_to(dual_model, dual_model_like)

# solve dual
MOI.optimize!(dual_model);

map = primal_dual_map.primal_con_dual_var;

μ   = MOI.get(dual_model, MOI.VariablePrimal(), map[con_index][1])

----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 30, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 11, constraints m = 18
Cones:	primal zero / dual free vars: 9
	soc vars: 3, soc blks: 1
	sd vars: 6, sd blks: 1
Setup time: 5.15e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.57e+00  4.55e+00  2.39e-01 -1.94e+01 -1.17e+01  0.00e+00  5.77e-05 
    40| 1.18e-08  9.41e-09  1.76e-09 -7.06e-01 -7.06e-01  2.46e-16  5.65e-04 
--------------------------------------

UndefVarError: UndefVarError: con_index not defined

In [51]:
_set_type(::MOI.ConstraintIndex{F,S}) where {F,S} = S

_set_type (generic function with 1 method)

In [53]:
cis = sort(
    model.con_idx, 
    by = x->CONES[_set_type(x)]
)

4-element Array{MathOptInterface.ConstraintIndex,1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.Zeros}(3)
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.Zeros}(4)
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(2)
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.PositiveSemidefiniteConeTriangle}(1)

In [55]:
cones = _set_type.(cis)

4-element Array{DataType,1}:
 MathOptInterface.Zeros
 MathOptInterface.Zeros
 MathOptInterface.SecondOrderCone
 MathOptInterface.PositiveSemidefiniteConeTriangle

In [58]:
unique!(cones)

3-element Array{DataType,1}:
 MathOptInterface.Zeros
 MathOptInterface.SecondOrderCone
 MathOptInterface.PositiveSemidefiniteConeTriangle

In [59]:
cones

3-element Array{DataType,1}:
 MathOptInterface.Zeros
 MathOptInterface.SecondOrderCone
 MathOptInterface.PositiveSemidefiniteConeTriangle